# AI-Assisted Café Rostering System

This notebook generates weekly schedules for a café using role-based constraints and employee skills.

## Features:
- Role-based scheduling (MANAGER, BARISTA, WAITER, SANDWICH)
- Flexible hours policy per role
- Weekend coverage with fallback logic
- Early morning SANDWICH shifts
- Comprehensive validation and reporting


## 1. Setup and Configuration


In [1]:
# Import required libraries
import pandas as pd
import datetime as dt
import sys
import importlib

# Import scheduler modules
from scheduler.config import load_config
from scheduler.data_io import read_employees, read_shifts, write_assignments
from scheduler.engine_baseline import greedy_schedule, build_requirements_for_day
from scheduler.validator import validate_assignments, summarize_assignments


In [2]:
# Configuration parameters
WEEK_ID = "2025-W36"  # Change this to generate different weeks
EMPLOYEES_CSV = "./data/employees_new_12w_v2.csv"
SHIFTS_CSV = "./data/shift_new_12w_v2.csv"
CONFIG_PATH = "./scheduler_config.yaml"
OUTPUT_CSV = "./shift_details_generated.csv"

print(f"Week ID: {WEEK_ID}")
print(f"Employees CSV: {EMPLOYEES_CSV}")
print(f"Shifts CSV: {SHIFTS_CSV}")
print(f"Config: {CONFIG_PATH}")
print(f"Output: {OUTPUT_CSV}")


Week ID: 2025-W36
Employees CSV: ./data/employees_new_12w_v2.csv
Shifts CSV: ./data/shift_new_12w_v2.csv
Config: ./scheduler_config.yaml
Output: ./shift_details_generated.csv


## 2. Load Data and Configuration


In [3]:
# Load configuration
cfg = load_config(CONFIG_PATH)
print("Configuration loaded successfully")
print(f"Timezone: {cfg.timezone}")
print(f"Default shift: {cfg.default_shift.start} - {cfg.default_shift.end}")
print(f"Hours policy: {cfg.hours_policy}")


Configuration loaded successfully
Timezone: Australia/Sydney
Default shift: 07:00 - 15:00
Hours policy: {'MANAGER': {'target_min': 38, 'target_max': 40, 'hard_cap': 40}, 'BARISTA': {'target_min': 16, 'target_max': 32, 'hard_cap': 38}, 'WAITER': {'target_min': 16, 'target_max': 32, 'hard_cap': 38}, 'SANDWICH': {'target_min': 16, 'target_max': 32, 'hard_cap': 36}}


In [4]:
# Load employees data
employees = read_employees(EMPLOYEES_CSV)
print(f"Loaded {len(employees)} employees")
print("\nEmployee roles:")
print(employees['primary_role'].value_counts())
print("\nSample employees:")
print(employees[['employee_id', 'first_name', 'last_name', 'primary_role']].head())


Loaded 8 employees

Employee roles:
primary_role
MANAGER     2
WAITER      2
BARISTA     2
SANDWICH    2
Name: count, dtype: int64

Sample employees:
   employee_id first_name last_name primary_role
0         1001        Max     Hayes      MANAGER
1         1002        Mia     Stone      MANAGER
2         1003      Wendy        Ng       WAITER
3         1004       Will     Brown       WAITER
4         1005      Bella      Tran      BARISTA


In [5]:
# Load shifts data
shifts = read_shifts(SHIFTS_CSV, week_id=WEEK_ID)

if shifts.empty:
    print(f"[WARN] No shifts found for {WEEK_ID}. Auto-creating a 7-day week.")
    year = int(WEEK_ID.split('-W')[0])
    week = int(WEEK_ID.split('-W')[1])
    dates = [dt.date.fromisocalendar(year, week, dow) for dow in range(1, 8)]
    
    # Create synthetic shift_ids that won't collide with CSV IDs
    base_id = 100000 + (year % 1000) * 100 + week
    syn = pd.DataFrame({
        'shift_id': [base_id + i for i in range(7)],
        'date': dates,
        'week_id': [WEEK_ID] * 7,
    })
    shifts = syn
    print(f"Created synthetic shifts for {WEEK_ID}")
else:
    print(f"[INFO] Using {len(shifts)} existing shifts for {WEEK_ID}")

print(f"\nShifts dates: {sorted(shifts['date'].unique())}")


[INFO] Using 7 existing shifts for 2025-W36

Shifts dates: [datetime.date(2025, 9, 1), datetime.date(2025, 9, 2), datetime.date(2025, 9, 3), datetime.date(2025, 9, 4), datetime.date(2025, 9, 5), datetime.date(2025, 9, 6), datetime.date(2025, 9, 7)]


## 3. Check Requirements and Feasibility


In [6]:
# Check daily requirements
print("Daily requirements:")
for date in sorted(shifts["date"].unique()):
    ds = pd.Timestamp(date).strftime("%Y-%m-%d")
    req = build_requirements_for_day(ds, cfg)
    day_name = pd.Timestamp(date).day_name()
    print(f"{ds} ({day_name}): {req}")

# Check employee availability by role
print("\nAvailable employees by role:")
role_counts = employees['primary_role'].value_counts()
for role in ['MANAGER', 'BARISTA', 'WAITER', 'SANDWICH']:
    count = role_counts.get(role, 0)
    print(f"{role}: {count} employees")


Daily requirements:
2025-09-01 (Monday): {'MANAGER': 1, 'BARISTA': 1, 'WAITER': 1, 'SANDWICH': 1}
2025-09-02 (Tuesday): {'MANAGER': 1, 'BARISTA': 1, 'WAITER': 1, 'SANDWICH': 1}
2025-09-03 (Wednesday): {'MANAGER': 1, 'BARISTA': 1, 'WAITER': 1, 'SANDWICH': 1}
2025-09-04 (Thursday): {'MANAGER': 1, 'BARISTA': 1, 'WAITER': 1, 'SANDWICH': 1}
2025-09-05 (Friday): {'MANAGER': 1, 'BARISTA': 1, 'WAITER': 1, 'SANDWICH': 1}
2025-09-06 (Saturday): {'MANAGER': 2, 'BARISTA': 1, 'WAITER': 2, 'SANDWICH': 1}
2025-09-07 (Sunday): {'MANAGER': 2, 'BARISTA': 1, 'WAITER': 2, 'SANDWICH': 1}

Available employees by role:
MANAGER: 2 employees
BARISTA: 2 employees
WAITER: 2 employees
SANDWICH: 2 employees


## 4. Generate Schedule


In [8]:
# Generate assignments using greedy algorithm with backtracking
print("[INFO] Generating assignments...")
try:
    assignments = greedy_schedule(employees, shifts, cfg)
    print(f"[SUCCESS] Generated {len(assignments)} assignments")
    
    # Save to CSV
    write_assignments(OUTPUT_CSV, assignments)
    print(f"[INFO] Assignments saved to {OUTPUT_CSV}")
    
except RuntimeError as e:
    print(f"[ERROR] Scheduling failed: {e}")
    print("This might be due to insufficient staff or conflicting constraints.")
    assignments = None


[INFO] Generating assignments...
[SUCCESS] Generated 32 assignments
[INFO] Assignments saved to ./shift_details_generated.csv


## 5. Validation


In [9]:
# Validate assignments if generation was successful
if assignments is not None and not assignments.empty:
    print("[INFO] Validating assignments...")
    
    # Build requirements by date for validation
    req_by_date = {}
    for date in sorted(shifts["date"].unique()):
        ds = pd.Timestamp(date).strftime("%Y-%m-%d")
        req_by_date[ds] = build_requirements_for_day(ds, cfg)

    try:
        validate_assignments(
            employees,
            shifts,
            assignments,
            start_hm=cfg.default_shift.start,
            end_hm=cfg.default_shift.end,
            requirements_by_date=req_by_date,
        )
        print("[OK] Validation passed - all assignments are valid!")
        
    except Exception as e:
        print(f"[ERROR] Validation failed: {e}")
        print("Check the detailed error message above to see what needs to be fixed.")
else:
    print("[SKIP] No assignments to validate")


[INFO] Validating assignments...
[ERROR] Validation failed: name 'per_role_caps' is not defined
Check the detailed error message above to see what needs to be fixed.


c:\Users\holyd\OneDrive\Рабочий стол\research project\scheduler\constraints.py:44: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(["emp_id", "date"]).apply(_overlap).any()


## 6. Create Comprehensive Schedule Report


In [10]:
# Create comprehensive CSV with all employee details
if assignments is not None and not assignments.empty:
    print("[INFO] Creating comprehensive schedule report...")
    
    # Merge assignments with full employee data
    detailed_assignments = assignments.merge(
        employees[['employee_id', 'first_name', 'last_name', 'primary_role', 
                  'skill_coffee', 'skill_sandwich', 'customer_service_rating', 'skill_speed']], 
        left_on='emp_id', right_on='employee_id', how='left'
    )

    # Convert times to datetime for better processing
    detailed_assignments['start_time'] = pd.to_datetime(detailed_assignments['start_time'])
    detailed_assignments['end_time'] = pd.to_datetime(detailed_assignments['end_time'])

    # Calculate shift duration in hours
    detailed_assignments['shift_hours'] = (
        detailed_assignments['end_time'] - detailed_assignments['start_time']
    ).dt.total_seconds() / 3600

    # Add day of week
    detailed_assignments['day_of_week'] = detailed_assignments['start_time'].dt.day_name()

    # Add date for easier filtering
    detailed_assignments['date'] = detailed_assignments['start_time'].dt.date

    # Reorder columns for better readability
    column_order = [
        'shift_id', 'emp_id', 'first_name', 'last_name', 'primary_role',
        'date', 'day_of_week', 'start_time', 'end_time', 'shift_hours',
        'skill_coffee', 'skill_sandwich', 'customer_service_rating', 'skill_speed'
    ]

    detailed_assignments = detailed_assignments[column_order]

    # Sort by date and start time
    detailed_assignments = detailed_assignments.sort_values(['date', 'start_time', 'emp_id'])

    # Save to comprehensive CSV
    comprehensive_output = 'comprehensive_schedule.csv'
    detailed_assignments.to_csv(comprehensive_output, index=False)
    print(f"[INFO] Comprehensive schedule saved to: {comprehensive_output}")
    print(f"Total assignments: {len(detailed_assignments)}")
    
    # Show sample of the comprehensive data
    print(f"\nSample of comprehensive schedule:")
    print(detailed_assignments.head(10))
    
else:
    print("[SKIP] No assignments to create comprehensive report")


[INFO] Creating comprehensive schedule report...
[INFO] Comprehensive schedule saved to: comprehensive_schedule.csv
Total assignments: 32

Sample of comprehensive schedule:
    shift_id  emp_id first_name last_name primary_role        date  \
15      1000    1007        Sam       Lee     SANDWICH  2025-09-01   
12      1000    1001        Max     Hayes      MANAGER  2025-09-01   
14      1000    1004       Will     Brown       WAITER  2025-09-01   
13      1000    1006        Ben      Park      BARISTA  2025-09-01   
19      1001    1007        Sam       Lee     SANDWICH  2025-09-02   
16      1001    1001        Max     Hayes      MANAGER  2025-09-02   
18      1001    1003      Wendy        Ng       WAITER  2025-09-02   
17      1001    1006        Ben      Park      BARISTA  2025-09-02   
23      1002    1008       Sara      Khan     SANDWICH  2025-09-03   
20      1002    1001        Max     Hayes      MANAGER  2025-09-03   

   day_of_week                start_time                

## 7. Employee Hours Analysis


In [11]:
# Analyze employee hours and workload distribution
if assignments is not None and not assignments.empty:
    print("[INFO] Analyzing employee hours...")
    
    # Calculate total hours per employee
    employee_hours = detailed_assignments.groupby(['emp_id', 'first_name', 'last_name', 'primary_role']).agg({
        'shift_hours': 'sum',
        'shift_id': 'count'
    }).rename(columns={'shift_id': 'total_shifts'}).reset_index()

    # Sort by role and then by hours
    employee_hours = employee_hours.sort_values(['primary_role', 'shift_hours'], ascending=[True, False])

    print("\n" + "="*80)
    print("EMPLOYEE HOURS SUMMARY")
    print("="*80)

    # Show hours by role
    for role in ['MANAGER', 'BARISTA', 'WAITER', 'SANDWICH']:
        role_employees = employee_hours[employee_hours['primary_role'] == role]
        if not role_employees.empty:
            print(f"\n{role} ROLE:")
            print("-" * 40)
            
            # Get role policy for comparison
            role_policy = cfg.hours_policy.get(role, {})
            target_min = role_policy.get('target_min', 0)
            target_max = role_policy.get('target_max', 40)
            hard_cap = role_policy.get('hard_cap', 40)
            
            for _, emp in role_employees.iterrows():
                hours = emp['shift_hours']
                shifts = emp['total_shifts']
                name = f"{emp['first_name']} {emp['last_name']}"
                
                # Check if within target range
                if target_min <= hours <= target_max:
                    status = "✓ Within target"
                elif hours < target_min:
                    status = f"⚠ Below target (need {target_min}h)"
                else:
                    status = f"⚠ Above target (max {target_max}h)"
                
                # Check hard cap
                if hours > hard_cap:
                    status = f"❌ OVER HARD CAP ({hard_cap}h)"
                
                print(f"  {name}: {hours:.1f}h ({shifts} shifts) - {status}")

    # Overall summary
    print(f"\n" + "="*80)
    print("OVERALL SUMMARY")
    print("="*80)

    total_hours = employee_hours['shift_hours'].sum()
    avg_hours = employee_hours['shift_hours'].mean()
    min_hours = employee_hours['shift_hours'].min()
    max_hours = employee_hours['shift_hours'].max()

    print(f"Total hours across all employees: {total_hours:.1f}h")
    print(f"Average hours per employee: {avg_hours:.1f}h")
    print(f"Hours range: {min_hours:.1f}h - {max_hours:.1f}h")

    # Check for any employees over hard caps
    over_cap = []
    for _, emp in employee_hours.iterrows():
        role = emp['primary_role']
        hours = emp['shift_hours']
        role_policy = cfg.hours_policy.get(role, {})
        hard_cap = role_policy.get('hard_cap', 40)
        
        if hours > hard_cap:
            over_cap.append(f"{emp['first_name']} {emp['last_name']} ({role}): {hours:.1f}h > {hard_cap}h")

    if over_cap:
        print(f"\n⚠️  EMPLOYEES OVER HARD CAP:")
        for emp in over_cap:
            print(f"  - {emp}")
    else:
        print(f"\n✅ All employees within their hard caps")
        
else:
    print("[SKIP] No assignments to analyze")


[INFO] Analyzing employee hours...

EMPLOYEE HOURS SUMMARY

MANAGER ROLE:
----------------------------------------
  Max Hayes: 40.0h (5 shifts) - ✓ Within target
  Mia Stone: 32.0h (4 shifts) - ⚠ Below target (need 38h)

BARISTA ROLE:
----------------------------------------
  Ben Park: 34.0h (5 shifts) - ⚠ Above target (max 32h)
  Bella Tran: 16.0h (2 shifts) - ✓ Within target

WAITER ROLE:
----------------------------------------
  Will Brown: 32.0h (5 shifts) - ✓ Within target
  Wendy Ng: 26.0h (4 shifts) - ✓ Within target

SANDWICH ROLE:
----------------------------------------
  Sam Lee: 31.0h (4 shifts) - ✓ Within target
  Sara Khan: 21.0h (3 shifts) - ✓ Within target

OVERALL SUMMARY
Total hours across all employees: 232.0h
Average hours per employee: 29.0h
Hours range: 16.0h - 40.0h

✅ All employees within their hard caps


## 8. Daily Schedule View


In [12]:
# Show daily schedule in a readable format
if assignments is not None and not assignments.empty:
    print("[INFO] Generating daily schedule view...")
    
    print("\n" + "="*80)
    print("DAILY SCHEDULE VIEW")
    print("="*80)

    # Group by date to show daily coverage
    for date in sorted(detailed_assignments['start_time'].dt.date.unique()):
        day_assignments = detailed_assignments[detailed_assignments['start_time'].dt.date == date]
        day_name = day_assignments.iloc[0]['start_time'].strftime('%A')
        date_str = day_assignments.iloc[0]['start_time'].strftime('%Y-%m-%d')
        
        print(f"\n{date_str} ({day_name})")
        print("-" * 30)
        
        # Group by role
        for role in sorted(day_assignments['primary_role'].unique()):
            role_assignments = day_assignments[day_assignments['primary_role'] == role]
            print(f"  {role}: {len(role_assignments)} person(s)")
            for _, shift in role_assignments.iterrows():
                name = f"{shift['first_name']} {shift['last_name']}"
                start_time = shift['start_time'].strftime('%H:%M')
                end_time = shift['end_time'].strftime('%H:%M')
                print(f"    - {name}: {start_time}-{end_time}")
                
else:
    print("[SKIP] No assignments to display")


[INFO] Generating daily schedule view...

DAILY SCHEDULE VIEW

2025-09-01 (Monday)
------------------------------
  BARISTA: 1 person(s)
    - Ben Park: 07:00-15:00
  MANAGER: 1 person(s)
    - Max Hayes: 07:00-15:00
  SANDWICH: 1 person(s)
    - Sam Lee: 05:00-12:00
  WAITER: 1 person(s)
    - Will Brown: 07:00-15:00

2025-09-02 (Tuesday)
------------------------------
  BARISTA: 1 person(s)
    - Ben Park: 07:00-15:00
  MANAGER: 1 person(s)
    - Max Hayes: 07:00-15:00
  SANDWICH: 1 person(s)
    - Sam Lee: 05:00-12:00
  WAITER: 1 person(s)
    - Wendy Ng: 07:00-15:00

2025-09-03 (Wednesday)
------------------------------
  BARISTA: 1 person(s)
    - Ben Park: 07:00-15:00
  MANAGER: 1 person(s)
    - Max Hayes: 07:00-15:00
  SANDWICH: 1 person(s)
    - Sara Khan: 05:00-12:00
  WAITER: 1 person(s)
    - Will Brown: 07:00-15:00

2025-09-04 (Thursday)
------------------------------
  BARISTA: 1 person(s)
    - Bella Tran: 07:00-15:00
  MANAGER: 1 person(s)
    - Mia Stone: 07:00-15:00
 

## 9. Summary and Next Steps


In [14]:
# Final summary
print("\n" + "="*80)
print("SCHEDULING COMPLETE")
print("="*80)

if assignments is not None and not assignments.empty:
    print(f"✅ Successfully generated {len(assignments)} assignments")
    print(f"📁 Basic schedule saved to: {OUTPUT_CSV}")
    print(f"📁 Comprehensive report saved to: comprehensive_schedule.csv")
    print(f"\n📊 Schedule covers 7 days")
    print(f"👥 {len(employees)} employees scheduled")
    
    # Show role distribution
    role_dist = detailed_assignments['primary_role'].value_counts()
    print(f"\n📈 Role distribution:")
    for role, count in role_dist.items():
        print(f"  {role}: {count} assignments")
    
    print(f"\n🎯 Next steps:")
    print(f"  1. Review the comprehensive_schedule.csv file")
    print(f"  2. Check employee hours are balanced")
    print(f"  3. Adjust config if needed and re-run")
    print(f"  4. Export to your scheduling system")
    
else:
    print("❌ Scheduling failed")
    print(f"\n🔧 Troubleshooting:")
    print(f"  1. Check if you have enough employees for each role")
    print(f"  2. Verify hours policy allows sufficient coverage")
    print(f"  3. Try reducing requirements in config")
    print(f"  4. Check if any employees are over their hard caps")

print(f"\n📝 Configuration used: {CONFIG_PATH}")
print(f"📅 Week generated: {WEEK_ID}")



SCHEDULING COMPLETE
✅ Successfully generated 32 assignments
📁 Basic schedule saved to: ./shift_details_generated.csv
📁 Comprehensive report saved to: comprehensive_schedule.csv

📊 Schedule covers 7 days
👥 8 employees scheduled

📈 Role distribution:
  MANAGER: 9 assignments
  WAITER: 9 assignments
  SANDWICH: 7 assignments
  BARISTA: 7 assignments

🎯 Next steps:
  1. Review the comprehensive_schedule.csv file
  2. Check employee hours are balanced
  3. Adjust config if needed and re-run
  4. Export to your scheduling system

📝 Configuration used: ./scheduler_config.yaml
📅 Week generated: 2025-W36


## 10. Employee Hours Check


In [17]:
# Check total hours worked by each employee for the generated week
if assignments is not None and not assignments.empty:
    print("[INFO] Checking employee hours for the generated week...")
    
    # Load the comprehensive schedule if it exists
    try:
        detailed_assignments = pd.read_csv("comprehensive_schedule.csv")
        print("✅ Loaded comprehensive schedule")
        # Ensure datetime conversion for loaded data
        detailed_assignments['start_time'] = pd.to_datetime(detailed_assignments['start_time'])
        detailed_assignments['end_time'] = pd.to_datetime(detailed_assignments['end_time'])
    except FileNotFoundError:
        print("⚠️ Comprehensive schedule not found, creating from assignments...")
        # Create detailed assignments from basic assignments
        detailed_assignments = assignments.merge(
            employees[['employee_id', 'first_name', 'last_name', 'primary_role']], 
            left_on='emp_id', right_on='employee_id', how='left'
        )
        
        # Convert times and calculate hours
        detailed_assignments['start_time'] = pd.to_datetime(detailed_assignments['start_time'])
        detailed_assignments['end_time'] = pd.to_datetime(detailed_assignments['end_time'])
        detailed_assignments['shift_hours'] = (
            detailed_assignments['end_time'] - detailed_assignments['start_time']
        ).dt.total_seconds() / 3600
    
    # Calculate total hours per employee
    employee_hours = detailed_assignments.groupby(['emp_id', 'first_name', 'last_name', 'primary_role']).agg({
        'shift_hours': 'sum',
        'shift_id': 'count'
    }).rename(columns={'shift_id': 'total_shifts'}).reset_index()
    
    # Sort by role and then by hours
    employee_hours = employee_hours.sort_values(['primary_role', 'shift_hours'], ascending=[True, False])
    
    print("\n" + "="*80)
    print("WEEKLY HOURS SUMMARY")
    print("="*80)
    
    # Show hours by role
    for role in ['MANAGER', 'BARISTA', 'WAITER', 'SANDWICH']:
        role_employees = employee_hours[employee_hours['primary_role'] == role]
        if not role_employees.empty:
            print(f"\n{role} ROLE:")
            print("-" * 50)
            
            # Get role policy for comparison
            role_policy = cfg.hours_policy.get(role, {})
            target_min = role_policy.get('target_min', 0)
            target_max = role_policy.get('target_max', 40)
            hard_cap = role_policy.get('hard_cap', 40)
            
            for _, emp in role_employees.iterrows():
                hours = emp['shift_hours']
                shifts = emp['total_shifts']
                name = f"{emp['first_name']} {emp['last_name']}"
                
                # Check if within target range
                if target_min <= hours <= target_max:
                    status = "✅ Within target"
                elif hours < target_min:
                    status = f"⚠️ Below target (need {target_min}h)"
                else:
                    status = f"⚠️ Above target (max {target_max}h)"
                
                # Check hard cap
                if hours > hard_cap:
                    status = f"❌ OVER HARD CAP ({hard_cap}h)"
                
                print(f"  {name}: {hours:.1f}h ({shifts} shifts) - {status}")
    
    # Overall summary
    print(f"\n" + "="*80)
    print("OVERALL WEEKLY SUMMARY")
    print("="*80)
    
    total_hours = employee_hours['shift_hours'].sum()
    avg_hours = employee_hours['shift_hours'].mean()
    min_hours = employee_hours['shift_hours'].min()
    max_hours = employee_hours['shift_hours'].max()
    
    print(f"Total hours across all employees: {total_hours:.1f}h")
    print(f"Average hours per employee: {avg_hours:.1f}h")
    print(f"Hours range: {min_hours:.1f}h - {max_hours:.1f}h")
    
    # Check for any employees over hard caps
    over_cap = []
    for _, emp in employee_hours.iterrows():
        role = emp['primary_role']
        hours = emp['shift_hours']
        role_policy = cfg.hours_policy.get(role, {})
        hard_cap = role_policy.get('hard_cap', 40)
        
        if hours > hard_cap:
            over_cap.append(f"{emp['first_name']} {emp['last_name']} ({role}): {hours:.1f}h > {hard_cap}h")
    
    if over_cap:
        print(f"\n⚠️ EMPLOYEES OVER HARD CAP:")
        for emp in over_cap:
            print(f"  - {emp}")
    else:
        print(f"\n✅ All employees within their hard caps")
    
    # Show detailed breakdown
    print(f"\n" + "="*80)
    print("DETAILED BREAKDOWN BY EMPLOYEE")
    print("="*80)
    
    for _, emp in employee_hours.iterrows():
        name = f"{emp['first_name']} {emp['last_name']}"
        role = emp['primary_role']
        hours = emp['shift_hours']
        shifts = emp['total_shifts']
        
        print(f"\n{name} ({role}):")
        print(f"  Total hours: {hours:.1f}h")
        print(f"  Number of shifts: {shifts}")
        print(f"  Average hours per shift: {hours/shifts:.1f}h")
        
        # Show individual shifts
        emp_shifts = detailed_assignments[detailed_assignments['emp_id'] == emp['emp_id']]
        for _, shift in emp_shifts.iterrows():
            date = shift['start_time'].strftime('%Y-%m-%d')
            day = shift['start_time'].strftime('%A')
            start_time = shift['start_time'].strftime('%H:%M')
            end_time = shift['end_time'].strftime('%H:%M')
            shift_hours = shift['shift_hours']
            print(f"    {date} ({day}): {start_time}-{end_time} ({shift_hours:.1f}h)")
    
else:
    print("[SKIP] No assignments to analyze")


[INFO] Checking employee hours for the generated week...
✅ Loaded comprehensive schedule

WEEKLY HOURS SUMMARY

MANAGER ROLE:
--------------------------------------------------
  Max Hayes: 40.0h (5 shifts) - ✅ Within target
  Mia Stone: 32.0h (4 shifts) - ⚠️ Below target (need 38h)

BARISTA ROLE:
--------------------------------------------------
  Ben Park: 34.0h (5 shifts) - ⚠️ Above target (max 32h)
  Bella Tran: 16.0h (2 shifts) - ✅ Within target

WAITER ROLE:
--------------------------------------------------
  Will Brown: 32.0h (5 shifts) - ✅ Within target
  Wendy Ng: 26.0h (4 shifts) - ✅ Within target

SANDWICH ROLE:
--------------------------------------------------
  Sam Lee: 31.0h (4 shifts) - ✅ Within target
  Sara Khan: 21.0h (3 shifts) - ✅ Within target

OVERALL WEEKLY SUMMARY
Total hours across all employees: 232.0h
Average hours per employee: 29.0h
Hours range: 16.0h - 40.0h

✅ All employees within their hard caps

DETAILED BREAKDOWN BY EMPLOYEE

Ben Park (BARISTA):
  T